In this notebook, we predict whether the health of a fetus is classified as normal, suspect, or pathological based on CTG data. We will practice Classification Algorithms to achieve the lowest prediction error.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("../input/fetal-health-classification/fetal_health.csv")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

#Checking null values

In [ ]:
train.isnull().sum()

#Analyze target column

In [ ]:
sns.countplot(x="fetal_health", data=train)

Above plot shows class imbalance.

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(train.corr(),annot=True, cmap="viridis")

Remove highly correlated features.

In [ ]:
train.drop(['histogram_median','histogram_mean', 'histogram_mode'], 1, inplace=True)

In [ ]:
X=train.drop(['fetal_health'], 1)

In [ ]:
y=train['fetal_health']

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# Importing train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
from sklearn import preprocessing
scaler=preprocessing.StandardScaler()
scaler=scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)


In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,VotingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

In [ ]:
classification_models = [
                         'DecisionTreeClassifier',
                         'RandomForestClassifier',
                         'XGBClassifier']

In [ ]:
cm = []
acc = []
models = []
estimators = []

In [ ]:
for classfication_model in classification_models:
    
    model = eval(classfication_model)()
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    models.append(type(model).__name__)
    estimators.append((type(model).__name__,model))
    cm.append(confusion_matrix(y_test,y_pred))
    acc.append(accuracy_score(y_test,y_pred))
    

In [ ]:
vc = VotingClassifier(estimators)
vc.fit(X_train,y_train)

In [ ]:
y_pred = vc.predict(X_test)
    
models.append(type(vc).__name__)

cm.append(confusion_matrix(y_test,y_pred))
acc.append(accuracy_score(y_test,y_pred))


In [ ]:
model_dict = {"Models":models,
             "CM":cm,
             "Accuracy":acc
             }

In [ ]:
model_df = pd.DataFrame.from_dict(model_dict,orient='index')
model_df.transpose()